In [ ]:
!pip install --upgrade prometheus-api-client

In [ ]:
from prometheus_api_client       import PrometheusConnect
from prometheus_api_client.utils import parse_datetime

In [ ]:
PROMETHEUS_URL   = '<PROMETHEUS_URL>'
PROMETHEUS_TOKEN = '<PROMETHEUS_TOKEN>'

In [ ]:
prometheus_connect = PrometheusConnect(
    url         = PROMETHEUS_URL,
    headers     = { 'Authorization' : f'bearer { PROMETHEUS_TOKEN }' },
    disable_ssl = True
)

In [ ]:
prometheus_connect.all_metrics()[:10]